In [222]:
%matplotlib inline
import numpy as np
import pandas as pd
from itertools import islice

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ndcg_score
import lightgbm as lgb

In [23]:
!ls -lh MQ2007/Fold1

total 41M
-r--r--r-- 1 gdanschin users 8.0M Jan 16  2009 test.txt
-r--r--r-- 1 gdanschin users  25M Jan 16  2009 train.txt
-r--r--r-- 1 gdanschin users 8.1M Jan 16  2009 vali.txt


In [67]:
def parse_dataset(path):
    data = pd.read_csv(path, header=None, sep=' ').iloc[:, :48]
    y = data.iloc[:, 0].values
    queries = data.iloc[:, 1].str.slice(4).astype(np.int).values
    X = []
    for column in range(2, data.shape[1]):
        X.append(data.iloc[:, column].str.slice(len(str(column - 1)) + 1).astype(np.float32).values)
    X = np.vstack(X).T
    return X, y, queries

In [89]:
def get_query_sizes(query_ids):
    _, serp_value_idx = np.unique(query_ids, return_index=True)
    return np.diff(np.concatenate([serp_value_idx, np.array([len(query_ids)])]))

In [364]:
def compute_ndcg(y_true, y_pred, q, k=None):
    query_sizes = get_query_sizes(q)
    start_idx = 0
    nDCG = []
    for query_size in query_sizes:
        nDCG.append(
            ndcg_score(y_true[start_idx:start_idx + query_size].reshape(1, -1),
                       y_pred[start_idx:start_idx + query_size].reshape(1, -1), k=k) 
        )
        start_idx += query_size
    return np.mean(nDCG)

In [387]:
def output_ndcg(y_true, y_pred, q):
    for k in [3, 5, 10, None]:
        score = compute_ndcg(y_true, y_pred, q, k=k)
        print('nDCG{}'.format(f'@{k}' if k is not None else '') + f' = {score:.5f}')

In [377]:
X_train, y_train, q_train = parse_dataset('MQ2007/Fold1/train.txt')
X_test, y_test, q_test = parse_dataset('MQ2007/Fold1/test.txt')

In [394]:
print(X_train.shape, y_train.shape, q_train.shape)

(42158, 46) (42158,) (42158,)


In [395]:
print(X_test.shape, y_test.shape, q_test.shape)

(13652, 46) (13652,) (13652,)


In [396]:
get_query_sizes(q_train).shape[0]

1017

In [397]:
get_query_sizes(q_test).shape[0]

336

In [398]:
output_ndcg(y_test, np.random.normal(size=y_test.shape[0]), q_test )

nDCG@3 = 0.24341
nDCG@5 = 0.25239
nDCG@10 = 0.28269
nDCG = 0.50617


In [399]:
%%time

query_sizes = get_query_sizes(q_train)
start_idx = 0

X_transformed = []
y_transformed = []

for query_size in query_sizes:
    X_query = X_train[start_idx:start_idx + query_size]
    y_query = y_train[start_idx:start_idx + query_size]
    if np.unique(y_query).shape[0] <= 1:
        continue
    
    X_query_transformed = []
    y_query_transformed = []
    
    for i in range(len(y_query)):
        for j in range(i + 1, len(y_query)):
            if y_query[i] != y_query[j]:
                diff = X_query[i] - X_query[j]
                X_query_transformed.append(diff)
                X_query_transformed.append(-diff)
                
                target = int(y_query[i] > y_query[j])
                y_query_transformed.append(target)
                y_query_transformed.append(1 - target)
    start_idx += query_size
    X_transformed.append(np.vstack(X_query_transformed))
    y_transformed.append(np.array(y_query_transformed))
    
X_transformed = np.vstack(X_transformed)
y_transformed = np.concatenate(y_transformed)

CPU times: user 664 ms, sys: 0 ns, total: 664 ms
Wall time: 663 ms


In [403]:
%%time
logreg = LogisticRegression(C=1.0, fit_intercept=False, max_iter=1000)
logreg.fit(X_transformed, y_transformed)

CPU times: user 2min 27s, sys: 5min 53s, total: 8min 20s
Wall time: 12.6 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [404]:
output_ndcg(y_test, logreg.predict_proba(X_test)[:, 1], q_test)

nDCG@3 = 0.42428
nDCG@5 = 0.43035
nDCG@10 = 0.45793
nDCG = 0.61394


In [405]:
%%time
rank_svm = LinearSVC(C=1.0, fit_intercept=False)
rank_svm.fit(X_transformed, y_transformed)

CPU times: user 45.6 s, sys: 40.6 ms, total: 45.7 s
Wall time: 45.7 s


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=False,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [406]:
output_ndcg(y_test, X_test.dot(rank_svm.coef_[0]), q_test)

nDCG@3 = 0.42538
nDCG@5 = 0.43319
nDCG@10 = 0.45894
nDCG = 0.61417


In [407]:
get_query_sizes(q_train)

array([40, 40, 40, ..., 11, 40, 40])

In [225]:
lgb_train = lgb.Dataset(X_train, label=y_train, group=get_query_sizes(q_train))
lgb_test = lgb.Dataset(X_test, label=y_test, group=get_query_sizes(q_test))

In [408]:
X_train

array([[0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.03131 , 0.666667, 0.5     , ..., 0.333333, 0.448276, 0.      ],
       [0.078682, 0.166667, 0.5     , ..., 0.833333, 0.678161, 0.      ],
       ...,
       [0.762295, 0.      , 0.      , ..., 0.5     , 0.686275, 0.      ],
       [0.02459 , 0.      , 0.      , ..., 0.5     , 0.352941, 0.      ],
       [0.663934, 0.      , 0.      , ..., 0.5     , 0.431373, 0.      ]],
      dtype=float32)

In [348]:
lgb_parameters = {
    'objective': 'lambdarank',
    'sigmoid': 1.0,
    'metric': ['ndcg'],
    'eta': 0.005,
    'max_depth': 4,
    'subsample': 1.0,
    'colsample_bytree': 0.75,
    'seed': 71014,
 
}

In [370]:
%%time
lgb_model = lgb.train(
    lgb_parameters, lgb_train, valid_sets=lgb_test, num_boost_round=5000,
)

[1]	valid_0's ndcg@1: 0.563492	valid_0's ndcg@2: 0.566619	valid_0's ndcg@3: 0.563366	valid_0's ndcg@4: 0.559827	valid_0's ndcg@5: 0.558606
[2]	valid_0's ndcg@1: 0.570437	valid_0's ndcg@2: 0.570056	valid_0's ndcg@3: 0.568767	valid_0's ndcg@4: 0.568092	valid_0's ndcg@5: 0.573759
[3]	valid_0's ndcg@1: 0.580357	valid_0's ndcg@2: 0.568659	valid_0's ndcg@3: 0.573959	valid_0's ndcg@4: 0.572594	valid_0's ndcg@5: 0.576373
[4]	valid_0's ndcg@1: 0.582341	valid_0's ndcg@2: 0.57202	valid_0's ndcg@3: 0.579168	valid_0's ndcg@4: 0.577633	valid_0's ndcg@5: 0.580464
[5]	valid_0's ndcg@1: 0.577381	valid_0's ndcg@2: 0.572525	valid_0's ndcg@3: 0.578743	valid_0's ndcg@4: 0.578398	valid_0's ndcg@5: 0.581035
[6]	valid_0's ndcg@1: 0.577381	valid_0's ndcg@2: 0.573255	valid_0's ndcg@3: 0.579515	valid_0's ndcg@4: 0.578408	valid_0's ndcg@5: 0.57941
[7]	valid_0's ndcg@1: 0.581349	valid_0's ndcg@2: 0.567629	valid_0's ndcg@3: 0.572979	valid_0's ndcg@4: 0.576279	valid_0's ndcg@5: 0.575318
[8]	valid_0's ndcg@1: 0.56944

[95]	valid_0's ndcg@1: 0.568452	valid_0's ndcg@2: 0.567469	valid_0's ndcg@3: 0.570659	valid_0's ndcg@4: 0.571113	valid_0's ndcg@5: 0.577358
[96]	valid_0's ndcg@1: 0.568452	valid_0's ndcg@2: 0.567469	valid_0's ndcg@3: 0.570299	valid_0's ndcg@4: 0.571401	valid_0's ndcg@5: 0.577346
[97]	valid_0's ndcg@1: 0.571429	valid_0's ndcg@2: 0.568142	valid_0's ndcg@3: 0.570815	valid_0's ndcg@4: 0.571496	valid_0's ndcg@5: 0.577689
[98]	valid_0's ndcg@1: 0.568452	valid_0's ndcg@2: 0.569905	valid_0's ndcg@3: 0.572095	valid_0's ndcg@4: 0.57108	valid_0's ndcg@5: 0.5766
[99]	valid_0's ndcg@1: 0.568452	valid_0's ndcg@2: 0.56862	valid_0's ndcg@3: 0.573275	valid_0's ndcg@4: 0.571542	valid_0's ndcg@5: 0.577576
[100]	valid_0's ndcg@1: 0.570437	valid_0's ndcg@2: 0.569225	valid_0's ndcg@3: 0.573807	valid_0's ndcg@4: 0.571856	valid_0's ndcg@5: 0.578498
[101]	valid_0's ndcg@1: 0.571429	valid_0's ndcg@2: 0.570985	valid_0's ndcg@3: 0.572826	valid_0's ndcg@4: 0.572066	valid_0's ndcg@5: 0.579565
[102]	valid_0's ndcg@1

[191]	valid_0's ndcg@1: 0.56746	valid_0's ndcg@2: 0.569252	valid_0's ndcg@3: 0.571992	valid_0's ndcg@4: 0.574905	valid_0's ndcg@5: 0.580038
[192]	valid_0's ndcg@1: 0.56746	valid_0's ndcg@2: 0.570019	valid_0's ndcg@3: 0.572114	valid_0's ndcg@4: 0.575019	valid_0's ndcg@5: 0.580146
[193]	valid_0's ndcg@1: 0.56746	valid_0's ndcg@2: 0.570019	valid_0's ndcg@3: 0.572114	valid_0's ndcg@4: 0.57552	valid_0's ndcg@5: 0.58045
[194]	valid_0's ndcg@1: 0.568452	valid_0's ndcg@2: 0.571011	valid_0's ndcg@3: 0.573339	valid_0's ndcg@4: 0.575881	valid_0's ndcg@5: 0.581645
[195]	valid_0's ndcg@1: 0.568452	valid_0's ndcg@2: 0.572163	valid_0's ndcg@3: 0.57422	valid_0's ndcg@4: 0.576614	valid_0's ndcg@5: 0.582282
[196]	valid_0's ndcg@1: 0.568452	valid_0's ndcg@2: 0.573314	valid_0's ndcg@3: 0.573704	valid_0's ndcg@4: 0.576686	valid_0's ndcg@5: 0.582734
[197]	valid_0's ndcg@1: 0.568452	valid_0's ndcg@2: 0.573314	valid_0's ndcg@3: 0.573472	valid_0's ndcg@4: 0.576656	valid_0's ndcg@5: 0.582706
[198]	valid_0's ndc

[291]	valid_0's ndcg@1: 0.564484	valid_0's ndcg@2: 0.573832	valid_0's ndcg@3: 0.569514	valid_0's ndcg@4: 0.571279	valid_0's ndcg@5: 0.579405
[292]	valid_0's ndcg@1: 0.56746	valid_0's ndcg@2: 0.574505	valid_0's ndcg@3: 0.57003	valid_0's ndcg@4: 0.571794	valid_0's ndcg@5: 0.57938
[293]	valid_0's ndcg@1: 0.56746	valid_0's ndcg@2: 0.574505	valid_0's ndcg@3: 0.571088	valid_0's ndcg@4: 0.57192	valid_0's ndcg@5: 0.580043
[294]	valid_0's ndcg@1: 0.566468	valid_0's ndcg@2: 0.574281	valid_0's ndcg@3: 0.570917	valid_0's ndcg@4: 0.571777	valid_0's ndcg@5: 0.579758
[295]	valid_0's ndcg@1: 0.566468	valid_0's ndcg@2: 0.574281	valid_0's ndcg@3: 0.570218	valid_0's ndcg@4: 0.571696	valid_0's ndcg@5: 0.578907
[296]	valid_0's ndcg@1: 0.563492	valid_0's ndcg@2: 0.573607	valid_0's ndcg@3: 0.569004	valid_0's ndcg@4: 0.571187	valid_0's ndcg@5: 0.578614
[297]	valid_0's ndcg@1: 0.563492	valid_0's ndcg@2: 0.573607	valid_0's ndcg@3: 0.568306	valid_0's ndcg@4: 0.571106	valid_0's ndcg@5: 0.578414
[298]	valid_0's nd

[388]	valid_0's ndcg@1: 0.565476	valid_0's ndcg@2: 0.572765	valid_0's ndcg@3: 0.568179	valid_0's ndcg@4: 0.571725	valid_0's ndcg@5: 0.576266
[389]	valid_0's ndcg@1: 0.565476	valid_0's ndcg@2: 0.573917	valid_0's ndcg@3: 0.568362	valid_0's ndcg@4: 0.571877	valid_0's ndcg@5: 0.576398
[390]	valid_0's ndcg@1: 0.565476	valid_0's ndcg@2: 0.573917	valid_0's ndcg@3: 0.56906	valid_0's ndcg@4: 0.571958	valid_0's ndcg@5: 0.576468
[391]	valid_0's ndcg@1: 0.565476	valid_0's ndcg@2: 0.573917	valid_0's ndcg@3: 0.569758	valid_0's ndcg@4: 0.572038	valid_0's ndcg@5: 0.576538
[392]	valid_0's ndcg@1: 0.565476	valid_0's ndcg@2: 0.573917	valid_0's ndcg@3: 0.569758	valid_0's ndcg@4: 0.572038	valid_0's ndcg@5: 0.577378
[393]	valid_0's ndcg@1: 0.566468	valid_0's ndcg@2: 0.574219	valid_0's ndcg@3: 0.569326	valid_0's ndcg@4: 0.572198	valid_0's ndcg@5: 0.578311
[394]	valid_0's ndcg@1: 0.566468	valid_0's ndcg@2: 0.574219	valid_0's ndcg@3: 0.56886	valid_0's ndcg@4: 0.572138	valid_0's ndcg@5: 0.577473
[395]	valid_0's

...

[4677]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.604235	valid_0's ndcg@3: 0.603764	valid_0's ndcg@4: 0.600174	valid_0's ndcg@5: 0.602394
[4678]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.604235	valid_0's ndcg@3: 0.603764	valid_0's ndcg@4: 0.600174	valid_0's ndcg@5: 0.602394
[4679]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.604235	valid_0's ndcg@3: 0.603764	valid_0's ndcg@4: 0.600174	valid_0's ndcg@5: 0.602394
[4680]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.604235	valid_0's ndcg@3: 0.603764	valid_0's ndcg@4: 0.600174	valid_0's ndcg@5: 0.602394
[4681]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.604235	valid_0's ndcg@3: 0.603764	valid_0's ndcg@4: 0.600174	valid_0's ndcg@5: 0.602394
[4682]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.604235	valid_0's ndcg@3: 0.603764	valid_0's ndcg@4: 0.600174	valid_0's ndcg@5: 0.602394
[4683]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.604235	valid_0's ndcg@3: 0.603764	valid_0's ndcg@4: 0.600174	valid_0's ndcg@5: 0.602394
[4684]

[4737]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.608114	valid_0's ndcg@3: 0.603964	valid_0's ndcg@4: 0.601237	valid_0's ndcg@5: 0.603016
[4738]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.608114	valid_0's ndcg@3: 0.603964	valid_0's ndcg@4: 0.601237	valid_0's ndcg@5: 0.603016
[4739]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.608114	valid_0's ndcg@3: 0.603964	valid_0's ndcg@4: 0.601237	valid_0's ndcg@5: 0.603016
[4740]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.608114	valid_0's ndcg@3: 0.603964	valid_0's ndcg@4: 0.601237	valid_0's ndcg@5: 0.603146
[4741]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.608114	valid_0's ndcg@3: 0.603964	valid_0's ndcg@4: 0.601237	valid_0's ndcg@5: 0.603146
[4742]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.608114	valid_0's ndcg@3: 0.603964	valid_0's ndcg@4: 0.601237	valid_0's ndcg@5: 0.603146
[4743]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.608114	valid_0's ndcg@3: 0.603964	valid_0's ndcg@4: 0.601237	valid_0's ndcg@5: 0.603146
[4744]

[4835]	valid_0's ndcg@1: 0.605159	valid_0's ndcg@2: 0.611023	valid_0's ndcg@3: 0.606608	valid_0's ndcg@4: 0.601378	valid_0's ndcg@5: 0.604751
[4836]	valid_0's ndcg@1: 0.605159	valid_0's ndcg@2: 0.611023	valid_0's ndcg@3: 0.606608	valid_0's ndcg@4: 0.601378	valid_0's ndcg@5: 0.604751
[4837]	valid_0's ndcg@1: 0.605159	valid_0's ndcg@2: 0.612175	valid_0's ndcg@3: 0.606791	valid_0's ndcg@4: 0.60153	valid_0's ndcg@5: 0.604883
[4838]	valid_0's ndcg@1: 0.605159	valid_0's ndcg@2: 0.613209	valid_0's ndcg@3: 0.60698	valid_0's ndcg@4: 0.601701	valid_0's ndcg@5: 0.605041
[4839]	valid_0's ndcg@1: 0.605159	valid_0's ndcg@2: 0.613209	valid_0's ndcg@3: 0.60698	valid_0's ndcg@4: 0.602034	valid_0's ndcg@5: 0.60507
[4840]	valid_0's ndcg@1: 0.605159	valid_0's ndcg@2: 0.612058	valid_0's ndcg@3: 0.606797	valid_0's ndcg@4: 0.601882	valid_0's ndcg@5: 0.604938
[4841]	valid_0's ndcg@1: 0.605159	valid_0's ndcg@2: 0.612058	valid_0's ndcg@3: 0.606797	valid_0's ndcg@4: 0.601882	valid_0's ndcg@5: 0.604938
[4842]	val

[4895]	valid_0's ndcg@1: 0.602183	valid_0's ndcg@2: 0.612536	valid_0's ndcg@3: 0.607163	valid_0's ndcg@4: 0.601686	valid_0's ndcg@5: 0.604174
[4896]	valid_0's ndcg@1: 0.602183	valid_0's ndcg@2: 0.612536	valid_0's ndcg@3: 0.607163	valid_0's ndcg@4: 0.601686	valid_0's ndcg@5: 0.604174
[4897]	valid_0's ndcg@1: 0.602183	valid_0's ndcg@2: 0.612536	valid_0's ndcg@3: 0.607163	valid_0's ndcg@4: 0.601686	valid_0's ndcg@5: 0.604174
[4898]	valid_0's ndcg@1: 0.602183	valid_0's ndcg@2: 0.612152	valid_0's ndcg@3: 0.607102	valid_0's ndcg@4: 0.601629	valid_0's ndcg@5: 0.60412
[4899]	valid_0's ndcg@1: 0.602183	valid_0's ndcg@2: 0.613687	valid_0's ndcg@3: 0.607346	valid_0's ndcg@4: 0.602339	valid_0's ndcg@5: 0.60435
[4900]	valid_0's ndcg@1: 0.602183	valid_0's ndcg@2: 0.613687	valid_0's ndcg@3: 0.607346	valid_0's ndcg@4: 0.602339	valid_0's ndcg@5: 0.60435
[4901]	valid_0's ndcg@1: 0.602183	valid_0's ndcg@2: 0.613687	valid_0's ndcg@3: 0.607346	valid_0's ndcg@4: 0.602339	valid_0's ndcg@5: 0.60435
[4902]	val

[4954]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.612376	valid_0's ndcg@3: 0.605667	valid_0's ndcg@4: 0.602333	valid_0's ndcg@5: 0.604893
[4955]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.612376	valid_0's ndcg@3: 0.605667	valid_0's ndcg@4: 0.6025	valid_0's ndcg@5: 0.604909
[4956]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.612376	valid_0's ndcg@3: 0.605667	valid_0's ndcg@4: 0.6025	valid_0's ndcg@5: 0.604909
[4957]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.612376	valid_0's ndcg@3: 0.605667	valid_0's ndcg@4: 0.6025	valid_0's ndcg@5: 0.605299
[4958]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.612376	valid_0's ndcg@3: 0.605667	valid_0's ndcg@4: 0.6025	valid_0's ndcg@5: 0.604909
[4959]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.612376	valid_0's ndcg@3: 0.605667	valid_0's ndcg@4: 0.6025	valid_0's ndcg@5: 0.604909
[4960]	valid_0's ndcg@1: 0.603175	valid_0's ndcg@2: 0.612376	valid_0's ndcg@3: 0.605667	valid_0's ndcg@4: 0.6025	valid_0's ndcg@5: 0.605299
[4961]	valid_0's n

In [391]:
output_ndcg(y_test, lgb_model.predict(X_test), q_test)

nDCG@3 = 0.48174
nDCG@5 = 0.48075
nDCG@10 = 0.50110
nDCG = 0.63916


In [ ]:
nDCG@3 = 0.42538
nDCG@5 = 0.43319
nDCG@10 = 0.45894
nDCG = 0.61417